# <font color='blue'>Técnicas de Reducción de Varianza


En las simulaciones Monte Carlo, como en otras técnicas de muestreo <font color='red'> el error es proporcional a $\frac{1}{\sqrt{N}}$, siendo $N$ el tamaño de la muestra.</font>

Si consideramos la **precisión** como proporcional a la desviación estándar, entonces se sigue que la cantidad de trabajo computacional requerido varía con el cuadrado de la precisión deseada.

Los **métodos de reducción de varianza** buscan reducir esa constante de proporcionalidad para aumentar la precisión sin tener que aumentar el trabajo computacional.

Entre esos métodos, podemos mencionar:

* Variables antitéticas.
* Muestreo Importancia.
* Muestreo estratificado. 
* Variables de control.
* Monte Carlo condicional.

## <font color='blue'> Variables Antitéticas

Esta técnica se basa en el uso del teorema:

$$Var(\frac{X_1+X_2}{2})=\frac{1}{4}[Var(X_1)+Var(X_1)+2Cov(X_1+X_2)]$$

Consideremos la estimación de la integral:

 <font color='red'> $$\mathcal{I}=\int_0^1 \frac{1}{1+x} dx$$

## Ejercicio:

* Calcular el valor exacto de esta integral.

Podemos estimar el valor de esta integral por el método de la media muestral:

$$\mathcal{I} \approx \frac{1}{1 + u_1} + \frac{1}{1 + u_2} + \frac{1}{1 + u_3} + ... + \frac{1}{1 + u_m}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Definimos la función de prueba.
def funcion(x):
    return 1.0/(1+x) 

In [ ]:
#Graficamos la función de prueba.
X=np.linspace(0,1,1000)
plt.plot(X,funcion(X), color='red', label="$y=1/(1+x)$")
#plt.fill_between(X,funcion(X))
plt.legend()
plt.xlim(0.0, 1.2)
plt.ylim(0.0, 1.2)
plt.grid(True)
plt.title('Función de prueba', color='b')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

Podemos estimar el valor de esta integral por el método de la media muestral:

$$\mathcal{I} \approx \frac{1}{1 + u_1} + \frac{1}{1 + u_2} + \frac{1}{1 + u_3} + ... + \frac{1}{1 + u_m}$$

In [ ]:
def integ(func=funcion, n=3000):
    muestreo=[]
    suma=[]
    for i in range(n):
        u= np.random.random_sample()
        muestreo.append(u)
        y=funcion(u)
        suma.append(y)

    return np.mean(suma), np.std(suma)

print("El valor de la integral es:", round(integ()[0],5))
print("El error estándar es:", round(integ()[1],5))

In [ ]:
def integ(func=funcion, n=3000):
    muestreo=[]
    suma=[]
    for i in range(n):
        u= np.random.random_sample()
        muestreo.append(u)
        y=funcion(1-u)
        suma.append(y)

    return np.mean(suma), np.std(suma), suma

print("El valor de la integral es:", round(integ()[0],5))
print("El error estándar es:", round(integ()[1],5))

In [ ]:
def integ(func=funcion, n=3000):
    muestreo=[]
    suma=[]
    for i in range(n):
        u= np.random.random_sample()
        muestreo.append(u)
        y=(funcion(u) + funcion(1-u))/2.0
        suma.append(y)

    return np.mean(suma), np.std(suma)

print("El valor de la integral es:", round(integ()[0],5))
print("El error estándar es:", round(integ()[1],5))

In [ ]:
def mc_crudo(funcion, n):
    integral=[]
    suma=[]
    for i in range(n):
        u= np.random.random_sample()
        y=(funcion(u) + funcion(1-u))/2.0
        suma.append(y)
        integral.append(np.mean(suma))
        
    print("El valor de la integral es:", round(np.mean(suma),5))
    print("El error estándar es:", round(np.std(suma),5))
    
    return np.mean(suma), np.std(suma), integral

In [ ]:
def mc_crudo1(funcion, n):
    integral=[]
    suma=[]
    for i in range(n):
        u= np.random.random_sample()
        y=funcion(u)
        suma.append(y)
        integral.append(np.mean(suma))
        
    print("El valor de la integral es:", round(np.mean(suma),5))
    print("El error estándar es:", round(np.std(suma),5))
    
    return np.mean(suma), np.std(suma), integral

In [ ]:
X1=mc_crudo1(funcion, n=1000)[2]
X=mc_crudo(funcion, n=1000)[2]

plt.plot(X, color='red', label='antiteticas')
plt.plot(X1, color='green', label='u')
plt.legend()

Supongamos que $\hat{\theta_1}$ es un estimador insesgado del párametro $\theta$ y que además $\hat{\theta_1}$ es una función de $m$ números aleatorios $R_1, ..., R_m$. Debido a que $1 - R_m$ sigue la misma distribución que $R_m$ (ambos $U(0,1)$), podemos construir otro estimador simplemente sustituyendo $R_m$ por $1-R_m$.  

## <font color='blue'> Jakknife

In [ ]:
import random
import statistics 
import matplotlib.pyplot as plt

In [ ]:
PopData = list()

random.seed(5)

for i in range(100):
    DataElem = 10 * random.random()
    PopData.append(DataElem)

In [ ]:
PopData

In [ ]:
def CVCalc(Dat):
    CVCalc = statistics.stdev(Dat)/statistics.mean(Dat) 
    return CVCalc

In [ ]:
CVPopData = CVCalc(PopData)
print(CVPopData)

In [ ]:
N = len(PopData)
JackVal = list()
PseudoVal = list()
for i in range(N-1):
    JackVal.append(0)
for i in range(N):
    PseudoVal.append(0)

for i in range(N):
    for j in range(N):
        if(j < i): 
            JackVal[j] = PopData[j]
        else:
            if(j > i):
                JackVal[j-1]= PopData[j]
    PseudoVal[i] = N*CVCalc(PopData)-(N-1)*CVCalc(JackVal)

In [ ]:
plt.hist(PseudoVal)
plt.show()

In [ ]:
MeanPseudoVal=statistics.mean(PseudoVal)
print(MeanPseudoVal)

In [ ]:
VariancePseudoVal=statistics.variance(PseudoVal)
print(VariancePseudoVal)

In [ ]:
MeanPseudoVal=statistics.mean(PseudoVal)
print(MeanPseudoVal)

## <font color='blue'> Bootstrap

In [ ]:
import random
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
PopData = list()

random.seed(7)

for i in range(1000):
    DataElem = 50 * random.random()
    PopData.append(DataElem)

In [ ]:
PopData

In [ ]:
PopSample = random.choices(PopData, k=100)

In [ ]:
PopSampleMean = list()
for i in range(10000):  
    SampleI = random.choices(PopData, k=100)
    PopSampleMean.append(np.mean(SampleI))

plt.hist(PopSampleMean)
plt.show()

In [ ]:
MeanPopSampleMean = np.mean(PopSampleMean)
print("The mean of the Bootstrap estimator is ",MeanPopSampleMean)

In [ ]:
MeanPopData = np.mean(PopData)
print("The mean of the population is ",MeanPopData)

In [ ]:
MeanPopSample = np.mean(PopSample)
print("The mean of the simple random sample is ",MeanPopSample)